## Moduli necessari

In [ ]:
import torch
from torch.utils import data
from os.path import join
from PIL import Image
import pandas as pd
import numpy as np
from torchvision import transforms
from torchvision.models import squeezenet1_0
from torchvision.models import SqueezeNet1_0_Weights
from torch import nn
import pickle
import os
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join
import os
import torch_directml  #Opzionale per usare qualsiasi GPU durante il training
import torch.optim as optim
from matplotlib import pyplot as plt
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True) #Rieseguire se si apportano modifiche all'env

## Classe per leggere immagini da CSV 
#### (Richiede CSV ricavati con dataset_maker_regression.py)

In [ ]:


class CSVImageDataset(data.Dataset):
    def __init__(self, data_root, csv, transform=None):
        self.data_root = data_root
        self.data = pd.read_csv(csv)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        im_path, im_label = self.data.iloc[i]['Path'], self.data.iloc[i].Label
        # il dataset contiene alcune immagini in scala di grigi
        # convertiamo tutto in RGB per avere delle immagini consistenti
        im = Image.open(join(self.data_root, im_path)).convert('RGB')
        
        #Per fare eventualmente data augmentation o quelle come toTensor?
        if self.transform is not None:
            im = self.transform(im)
        
        return im, im_label

## Calcolo media e std per normalizzazione

In [ ]:
def salva_valori_su_file(valori, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(valori, file)

def leggi_valori_da_file(file_path):
    with open(file_path, 'rb') as file:
        valori = pickle.load(file)
    return valori

In [ ]:
resolution=640
os.makedirs("bin",exist_ok=True)
file_path = 'valori.pkl'

#******** SE ESISTE UN FILE CON VALORI PRECALCOLATI USA QUELLI********
if os.path.exists(file_path):
    valori = leggi_valori_da_file(file_path)
    print("Valori letti dal file:", valori)
    red_mean=valori[0]
    green_mean=valori[1]
    blue_mean=valori[2]
    red_std=valori[3]
    green_std=valori[4]
    blue_std=valori[5]
else:

    preprocessing_trasnform = transforms.Compose([
    transforms.Resize((resolution,resolution)),
    transforms.ToTensor(),
    ])

    dataset_train=CSVImageDataset('euro/train/images','train.csv',transform=preprocessing_trasnform)
    dataset_valid=CSVImageDataset('euro/valid/images','valid.csv',transform=preprocessing_trasnform)
    dataset_test=CSVImageDataset('euro/test/images','test.csv',transform=preprocessing_trasnform)

    red_mean=0
    green_mean=0
    blue_mean=0
    num_pixel=(len(dataset_train)*(resolution)**2)
    for image in dataset_train:
        red_mean+=image[0][0].sum()# somma di tutti i pixel
        green_mean+=image[0][1].sum()# somma di tutti i pixel
        blue_mean+=image[0][2].sum()# somma di tutti i pixel
    #dividiamo per il numero di immagini molt numero di pixel
    red_mean=red_mean/num_pixel
    green_mean=green_mean/num_pixel
    blue_mean=blue_mean/num_pixel
    #deviazione standard
    red_std=0
    green_std=0
    blue_std=0
    for image in dataset_train:
        red_std+=((image[0][0]-red_mean)**2).sum()
        green_std+=((image[0][1]-green_mean)**2).sum()
        blue_std+=((image[0][2]-blue_mean)**2).sum()
    #sqrt della varianza
    red_std=np.sqrt(red_std/num_pixel)
    green_std=np.sqrt(green_std/num_pixel)
    blue_std=np.sqrt(blue_std/num_pixel)

    valori=[red_mean,green_mean,blue_mean,red_std,green_std,blue_std]
    salva_valori_su_file(valori, file_path)
    print("Valori calcolati e salvati:", valori)

## Preparazione del modello

In [ ]:
# model= squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)
model= squeezenet1_0(weights=None) # Non facciamo Fine-tuning, utilizziamo solo il modello

num_class=1
model.num_classes=num_class

# Vecchio blocco per fare classificazione su 1000 classi
# classifier= nn.Sequential(
#     nn.Dropout(p=0.5),
#     nn.Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1)),
#     nn.ReLU(),
#     nn.AdaptiveAvgPool2d((1, 1)),
# )


# Creazione di un nuovo blocco "classifier" ma che si adatti al task di regressione
new_classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),  # Appiattisce l'output da (256, 1, 1) a (256,1)
    nn.Linear(256, num_class)  # Layer lineare per la regressione
)

# Sostituzione del vecchio classifier con il nuovo
model.classifier = new_classifier

### DataLoader

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(640),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([red_mean, green_mean, blue_mean], [red_std, green_std, blue_std])
])

test_transform = transforms.Compose([
    transforms.Resize(640),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([red_mean, green_mean, blue_mean], [red_std, green_std, blue_std])
])

dataset_train=CSVImageDataset('euro/train/images','train.csv',transform=train_transform)
dataset_valid=CSVImageDataset('euro/valid/images','valid.csv',transform=test_transform)
dataset_test=CSVImageDataset('euro/test/images','test.csv',transform=test_transform)

train_loader=(DataLoader(dataset_train,batch_size=16,num_workers=0,shuffle=True))
valid_loader=(DataLoader(dataset_valid,batch_size=16,num_workers=0))
test_loader=(DataLoader(dataset_test,batch_size=16,num_workers=0))

im,lab=dataset_train[150]
print('#Coins:',lab)
print(im.shape)
im.mean()
im[0].mean()

## Classe per calcolare la media delle loss

In [ ]:
class AverageValueMeter():
    def init(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, value, num):
        self.sum += value*num
        self.num += num

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None

## Funzione per effettuare il training del modello

In [ ]:
def trainval(model, train_loader, valid_loader, exp_name='experiment', lr=0.01, epochs=150, momentum=0.99, logdir='logs'):
    criterion = nn.MSELoss() #Per regressione usiamo l'MSE loss

    #optimizer = SGD(model.parameters(), lr, momentum=momentum)
    optimizer=optim.Adam(model.parameters(), lr=lr)

    # Meters
    loss_meter = AverageValueMeter()
    
    # Writer
    writer = SummaryWriter(join(logdir, exp_name))
    
    # Device
    #*****DIRECTML*****
    # dml=torch_directml.device()
    # device = "cuda" if torch.cuda.is_available() else  dml
    # model.to(device)
    #******************

    device = "cuda" if torch.cuda.is_available() else  "cpu"
    model.to(device)

    # Definiamo un dizionario contenente i loader di training e test
    loader = {
        'train': train_loader,
        'valid': valid_loader
    }

    # Inizializziamo il global step (Num. immagini viste)
    global_step = 0
    # Creiamo la cartella dove salvare il modello se non esiste già
    os.makedirs("checkpoints", exist_ok=True)

    for e in range(epochs):
        print(f"Epoch {e+1} of {epochs}")
        # iteriamo tra due modalità: train e test
        for mode in ['train', 'valid']:

            loss_meter.reset()
            model.train() if mode == 'train' else model.eval() # Modalità eval durante la fase di validation

            with torch.set_grad_enabled(mode=='train'): # Abilitiamo i gradienti solo in training
                for i, batch in enumerate(loader[mode]):
                    x = batch[0].to(device) # Portiamoli sul device corretto
                    y = batch[1].to(device)

                    y=y.float()

                    output = model(x)
                    n = x.shape[0] # numero di elementi nel batch
                    global_step += n
                    l = criterion(output.view(-1), y)
                    
                    if mode=='train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                    
                    loss_meter.add(l.item(), n)
                    
                    #**************DISATTIVATO=>LOGGHIAMO SOLO A FINE EPOCA
                    # Loggiamo i risultati iterazione per iterazione solo durante il training
                    #if mode=='train':
                    #    writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
            
            # Una volta finita l'epoca (sia nel caso di training che test, loggiamo le stime finali)
            writer.add_scalar('loss/' + mode, loss_meter.value(), global_step=e+1)
            
        #Salviamo un checkpoint alla fine di ogni epoca
        torch.save({
            'epoch': e,
            'model': model,
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step':global_step
            }, './checkpoints/%s-%d.pth'%(exp_name, e+1))
    return model

In [ ]:
squeezenet_regression=trainval(model,train_loader,valid_loader,exp_name='prova',lr=0.0001,epochs=10)

### Per testare il modello utilizzare regression_test.ipynb